In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 755.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.3 MB/s eta 0:00:00


In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [4]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [5]:
from langchain import PromptTemplate,  LLMChain

template = """
              Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
           """

prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm)


In [8]:
text1 = """
OpenAI founder Sam Altman’s sprawling network of investments
Altman has invested in hundreds of start-ups, adding to his massive influence in Silicon Valley
Sam Altman is best known as the chief executive of OpenAI — the pioneering artificial intelligence company that has turned him into one of Silicon Valley’s most powerful people. But before that, Altman was known as one of the tech industry’s most prolific angel investors, making dozens of investments in a range of companies focused on extending life to building fusion reactors.
Since 2010, Altman has personally invested in 125 companies, according to venture capital research firm PitchBook. Other entities that he’s associated with, including an investment firm he founded with his brothers called Hydrazine Capital, have invested in two dozen more. This year alone, as OpenAI grew rapidly under his leadership, he made more than 20 investments. From 2011 to 2019, Altman was also an investor at Y Combinator, Silicon Valley’s most prominent start-up incubator, where he served as lead partner on another 280 deals, giving him influence among dozens of the most promising start-ups in tech.
Here are a few of the most notable investments in Altman’s empire.
Helion Energy
Nuclear fusion has been a science-fiction fantasy for decades. Unlike regular nuclear reactors, which generate power by splitting atoms, fusion reactors do the same by smashing two atoms together. A recent breakthrough from U.S. government scientists has ignited excitement that clean and cheap fusion energy may actually be possible. Despite those recent advances, many scientists and engineers are still skeptical that fusion will work any time soon. Several start-ups are trying to do it anyway, including Helion Energy. Altman invested in the Everett, Wash.-based company as a partner at Y Combinator in 2014. Then in 2021, he invested $375 million in the company, according to venture capital data firm CB Insights, his largest personal investment ever.
In May, just months after investing billions of dollars in OpenAI, Microsoft signed a deal to be the first company to buy electricity from Helion, which it expects to begin producing in 2028.
Boom Aviation
Remember the Concorde jet? Boom Aviation is one of a small group of companies trying to bring back supersonic commercial travel more than 20 years after the Concorde made its last flight. Altman first invested in Boom as a private investor in 2016. Then, as a partner at Y Combinator in 2019, Altman invested in the company again. Boom has signed tentative purchase agreements for its still-under-development Overture jet with United and American Airlines and it plans to have the jets carrying passengers by 2029.
Retro Biosciences
The biotechnology company wants to slow aging by developing techniques to remove damaged cells and treating older cells to make them act as if they were young again. Altman invested $180 million in the company in 2022. Along with Helion, the two huge investments accounted for “all my liquid net worth,” Altman told MIT Technology Review earlier this year. Despite the huge cash infusion, the company is still very early-stage, announcing its presence to the world in 2022 when Altman made his investment.
"""
print(llm_chain.run(text1))


 - Altman has invested in over 125 companies since 2010, with an additional 25 investments from entities he is associated with.
            - He made over 20 investments this year alone, as OpenAI grew rapidly under his leadership.
            - Altman invested $375 million in Helion Energy in 2021, his largest personal investment ever.
            - Microsoft signed a deal to be the first company to buy electricity from Helion, expected to begin producing in 2028.
            - Altman first invested in Boom Aviation in 2016 and again in 2019 as a partner at Y Combinator.
            - Boom has signed tentative purchase agreements for its still-under-development Overture jet with United and American Airlines, with plans to have the jets carrying passengers by 2029.
            - Altman invested $180 million in Retro Biosciences in 2022, accounting for "all my liquid net worth".
            - Despite the huge cash infusion, Retro Biosciences is still very early-stage, announcing its pre

In [9]:
template = """
              Detect named entities in following text delimited by triple backquotes.
              Return your response in json format with spans of named entities with fields "named entity","type","span".
              Return all entities
              ```{text}```
              json format file:
           """

prompt = PromptTemplate(template=template, input_variables=["text1"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [10]:
print(llm_chain.run(text1))

 {
  "namedEntities": [
    {
      "namedEntity": "Sam Altman",
      "type": "Person",
      "span": [
        25,
        32
      ]
    },
    {
      "namedEntity": "OpenAI",
      "type": "Organization",
      "span": [
        34,
        44
      ]
    },
    {
      "namedEntity": "Helion Energy",
      "type": "Organization",
      "span": [
        45,
        55
      ]
    },
    {
      "namedEntity": "Boom Aviation",
      "type": "Organization",
      "span": [
        56,
        66
      ]
    },
    {
      "namedEntity": "Retro Biosciences",
      "type": "Organization",
      "span": [
        67,
        77
      ]
    }
  ]
}

Note: The above response is just an example, the actual response will have more entities and their spans.
Also, the named entities in the text are highlighted in bold.


# New Section